# Trp-cage haMSM Construction and Analysis 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jdrusso/msm_we/HEAD?labpath=examples%2Fhamsm_construction.ipynb)

For these examples, we'll be constructing an haMSM from simulations of Trp-cage unfolding.

In [1]:
from msm_we import modelWE
import numpy as np
import ray

In [2]:
import pickle

from msm_we._logging import log
# This is just to clean up the logging output for display in the documentation webpage.
log.handlers[0]._log_render.show_time = False
log.handlers[0].console.width = 65

# Set to True to re-generate reference files for unit tests.
generate_test_files = False

## Prep

First, let's set some parameters for haMSM building.

In [3]:
h5file_paths = ['../tests/reference/1000ns_ntl9/west.h5']

# Number of MSM microstates to initially put in each stratum/WE bin
clusters_per_stratum = 25

dimreduce_method = 'pca'

# Boundaries of the basis/target, in progress coordinate space
pcoord_bounds = {
    'basis': [[0, 0.15]],
    'target': [[0.7, 100]]
}

model_name = 'NaCl Sample'

# Reference structure
ref_file = 'data/2JOF.pdb'

# WESTPA resampling time
tau = 1e-9

### Define MSM featurization

The function `processCoordinates` defines a transformation that's applied to the coordinates in `auxdata/coord` before dimensionality reduction/clustering.

The inputs are the coordinates (read from `auxdata/coord`) for each segment in the loaded iteration(s), and it should return an array of features for each.

We could just use full XYZ coordinates for our MSM features. 
This would be a very simple (and very poor) choice -- it doesn't capture some of the translational invariance that something like pairwise distances might.

In [4]:
def processCoordinates(self, coords):
    
    return coords.reshape(coords.shape[0], -1)
    
modelWE.processCoordinates = processCoordinates

So, let's use pairwise distances instead, at the cost of a more expensive calculation.

Note that because of how work is serialized for parallelization, we have to define the `Universe` objects *within* `processCoordinates`. Otherwise, there would be multiple workers acting on the same object.

In [5]:
import MDAnalysis as mda
from MDAnalysis.analysis import distances

def processCoordinates(self, coords):
    
    u_ref = mda.Universe(ref_file)
    u_check = mda.Universe(ref_file)
    
    dist_out = []
    
    u_check.load_new(coords)

    for frame in u_check.trajectory:

        dists = distances.dist(
            u_check.select_atoms('backbone'),
            u_ref.select_atoms('backbone')
        )[2]

        dist_out.append(dists)

    dist_out = np.array(dist_out)
    
    return dist_out
    
modelWE.processCoordinates = processCoordinates

## Model-building with [build_analyze_model()](../api.rst#msm_we.modelWE.build_analyze_model)

In [6]:
model = modelWE()

model.build_analyze_model(
    file_paths=h5file_paths,
    # ref_struct=basis_ref_dict,
    ref_struct=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dimreduce_method=dimreduce_method,
    n_clusters=clusters_per_stratum,
    tau=tau
)

Output()

2022-10-10 10:41:57,299 INFO worker.py:1509 -- Started a local 
Ray instance. View the dashboard at 127.0.0.1:8265 

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

Initial iPCA:   0%|          | 0/50 [00:00<?, ?it/s]

iPCA:   0%|          | 0/99 [00:00<?, ?it/s]

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained             ]8;id=249619;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py\_fluxmatrix.py]8;;\:]8;id=783216;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py#497\497]8;;\

INFO                                             ]8;id=946439;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=362826;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#170\170]8;;\
         Flux converged to 2.8846e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.      ]8;id=870649;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=806072;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#181\181]8;;\

## Model-building step by step

[build_analyze_model()](../api.rst#msm_we.modelWE.build_analyze_model) is just a convenient wrapper around the following steps.

You can run them manually if you want to observe each step of the process.

It's helpful to start off by running step-by-step while you're starting analysis for a system, to fine-tune parameters without re-running the entire workflow.

First, we launch Ray, which is the backend for the parallel work manager.

In [7]:
ray.init(ignore_reinit_error=True)

2022-10-10 10:42:47,151 INFO worker.py:1351 -- Calling ray.init()
again after it has already been called.

Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [8]:
model = modelWE()

In [9]:
model.initialize(
    fileSpecifier=['../tests/reference/1000ns_ntl9/west.h5'],
    refPDBfile=ref_file,
    modelName=model_name,
    basis_pcoord_bounds=pcoord_bounds['basis'],
    target_pcoord_bounds=pcoord_bounds['target'],
    dim_reduce_method=dimreduce_method,
    tau=tau
)

In [10]:
if generate_test_files:
    with open('../tests/reference/1000ns_ntl9/models/initialized.obj', 'wb') as outfile:
        pickle.dump(model, outfile)

In [11]:
model.get_iterations()
model.get_coordSet(last_iter = model.maxIter, streaming=True)

Getting coordSet:   0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
if generate_test_files:
    with open('../tests/reference/1000ns_ntl9/models/loaded.obj', 'wb') as outfile:
        pickle.dump(model, outfile)

In [13]:
model.dimReduce()

Initial iPCA:   0%|          | 0/50 [00:00<?, ?it/s]

iPCA:   0%|          | 0/99 [00:00<?, ?it/s]

`store_validation_model=True` is necessary for block validation. It enables caching the model immediately after discretization in `model.post_cluster_model`, so that the blocks can be efficiently created later.

In [14]:
model.cluster_coordinates(
    n_clusters=clusters_per_stratum,
    use_ray=True,
    stratified=True,
    store_validation_model=True, # Required for block validation
    random_state=1337,
)

INFO     Be aware: Number of cluster centers   ]8;id=615963;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=707001;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#141\141]8;;\
         is an important parameter, and can                      
         drastically affect model quality. We                    
         recommend examining block-validation                    
         results with a range of numbers of                      
         clusters, to check for overfitting.                     

INFO     Beginning stratified clustering.      ]8;id=221524;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=144896;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#148\148]8;;\

INFO     Obtaining bin definitions from        ]8;id=357201;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=898692;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#568\568]8;;\
         iteration 2 in file                                     
         ../tests/reference/1000ns_ntl9/west.h                   
         5                                                       

INFO     Loading pickled bin mapper from H5    ]8;id=236491;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=209402;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#572\572]8;;\
         for stratified clustering...                            

ERROR    Stratified clustering currently MUST  ]8;id=288694;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=705837;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#608\608]8;;\
         run in streaming mode (and with ray                     
         enabled for discretization). Enabling                   
         both and continuing...                                  

...

Clustering:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=4522;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=837895;file:///home/jd/msm_we/msm_we/msm_we.py#450\450]8;;\

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete              ]8;id=207800;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=821006;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#1204\1204]8;;\

In [15]:
if generate_test_files:
    with open('../tests/reference/1000ns_ntl9/models/clustered.obj', 'wb') as outfile:
        pickle.dump(model, outfile)

In [16]:
model.get_fluxMatrix(n_lag=0)

Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

In [17]:
if generate_test_files:
    np.save('../tests/reference/1000ns_ntl9/models/fluxmatrix_raw.npy', model.fluxMatrixRaw)
    with open('../tests/reference/1000ns_ntl9/models/fluxmatrixed.obj', 'wb') as outfile:
        pickle.dump(model, outfile)

It's likely our flux matrix isn't strongly connected. This cleaning step removes all clusters that aren't in the largest connected set, then rediscretizes all the trajectories according to the new, reduced set of clusters.

In [18]:
model.organize_fluxMatrix()

INFO     Started with 300 clusters, and       ]8;id=45726;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=970302;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#1020\1020]8;;\
         removed 109                                             

WARNING  All clusters were cleaned from bins  ]8;id=480370;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=136604;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#1036\1036]8;;\
         {11}                                                    

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=253493;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=172343;file:///home/jd/msm_we/msm_we/msm_we.py#450\450]8;;\

INFO     Using cached model for               ]8;id=585129;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=22266;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#1132\1132]8;;\
         discretization                                          

Submitting discretization tasks:   0%|          | 0/99 [00:00<?, ?it/s]

Retrieving discretized trajectories:   0%|          | 0/99 [00:00<?, ?it/s]

INFO     Discretization complete              ]8;id=765104;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=277378;file:///home/jd/msm_we/msm_we/_hamsm/_clustering.py#1204\1204]8;;\

Constructing flux matrix:   0%|          | 0/98 [00:00<?, ?it/s]

In [19]:
if generate_test_files:
    with open('../tests/reference/1000ns_ntl9/models/organized.obj', 'wb') as outfile:
        pickle.dump(model, outfile)
    np.save('../tests/reference/1000ns_ntl9/models/fluxmatrix.npy', model.fluxMatrix)

### Transition matrix estimation

In [20]:
model.get_Tmatrix()

In [21]:
model.Tmatrix

array([[0.32293215, 0.30579927, 0.01005867, ..., 0.        , 0.33022179,
        0.        ],
       [0.27769355, 0.31023711, 0.0148812 , ..., 0.        , 0.2590533 ,
        0.        ],
       [0.28155956, 0.24801922, 0.00295366, ..., 0.        , 0.26514922,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.32410505],
       [0.31693347, 0.27896914, 0.0031783 , ..., 0.        , 0.35730547,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [22]:
model.get_steady_state()

INFO                                             ]8;id=50995;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=284965;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#170\170]8;;\
         Flux converged to 2.0346e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.      ]8;id=793862;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=940546;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#181\181]8;;\

In [23]:
model.get_steady_state_target_flux()

In [24]:
if generate_test_files:
    np.save('../tests/reference/1000ns_ntl9/models/tmatrix.npy', model.Tmatrix)
    np.save('../tests/reference/1000ns_ntl9/models/pSS.npy', model.pSS)
    np.save('../tests/reference/1000ns_ntl9/models/JtargetSS.npy', model.JtargetSS)

### Model validation

This block validation step 

In [25]:
model.do_block_validation(
    cross_validation_groups=2, 
    cross_validation_blocks=4
)

INFO     Beginning analysis of cross-validation     ]8;id=556310;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=633227;file:///home/jd/msm_we/msm_we/msm_we.py#926\926]8;;\
         group 1/2.                                              

INFO     Using Ray cluster with 16.0 CPUs!          ]8;id=390983;file:///home/jd/msm_we/msm_we/msm_we.py\msm_we.py]8;;\:]8;id=637012;file:///home/jd/msm_we/msm_we/msm_we.py#450\450]8;;\

Submitting fluxmatrix tasks:   0%|          | 0/50 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/50 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained             ]8;id=677172;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py\_fluxmatrix.py]8;;\:]8;id=96719;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py#497\497]8;;\

...

Submitting fluxmatrix tasks:   0%|          | 0/49 [00:00<?, ?it/s]

Retrieving flux matrices:   0%|          | 0/49 [00:00<?, ?it/s]

INFO     Fluxmatrices all obtained             ]8;id=346693;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py\_fluxmatrix.py]8;;\:]8;id=347789;file:///home/jd/msm_we/msm_we/_hamsm/_fluxmatrix.py#497\497]8;;\

INFO                                             ]8;id=205473;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=349337;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#170\170]8;;\
         Flux converged to 4.4589e+06 after 1                    
         iterations of inverse iteration.                        

INFO     Done with steady-state estimation.      ]8;id=762742;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=576163;file:///home/jd/msm_we/msm_we/_hamsm/_analysis.py#181\181]8;;\

### Generate lists of structures in each cluster

In [26]:
model.update_cluster_structures()

In [27]:
if generate_test_files:
    np.save('../tests/reference/1000ns_ntl9/models/bin10_cluster_structures.npy', model.cluster_structures[10])
    with open('../tests/reference/1000ns_ntl9/models/completed.obj', 'wb') as outfile:
        pickle.dump(model, outfile)

## Save model

In [28]:
with open('data/pickled_model', 'wb') as of:
    pickle.dump(model, of)